# Creación de APIs con FastAPI
El objetivo de esta sección es crear una API para consumir uno de los modelos que se generaron en el workshop anterior "scikit-learn y MLFlow". De igual forma, se empleará el pipeline de preprocesamiento de datos que se creó en el workshop anterior  "Pipelines en scikit-learn". 

Antes de arrancar, aseguremosnos de tener las siguientes dependencias en nuestro entorno de desarrollo:

```
dill==0.3.3
scikit-learn==0.24.1
python-multipart==0.0.5
numpy==1.21.0
pandas==1.2.5
matplotlib==3.4.2
mlflow==1.20.2
seaborn==0.11.1
fastapi==0.68.1
uvicorn==0.15.0
```

In [1]:
import dill
dill.__version__

'0.3.3'

In [2]:
import matplotlib
matplotlib.__version__

'3.4.2'

In [3]:
import mlflow
mlflow.__version__

'1.20.2'

In [4]:
import numpy
numpy.__version__

'1.20.3'

In [5]:
import pandas
pandas.__version__

'1.2.5'

In [6]:
import sklearn
sklearn.__version__

'0.24.2'

In [7]:
import seaborn
seaborn.__version__

'0.11.1'

In [8]:
import fastapi
fastapi.__version__

'0.68.1'

In [9]:
import uvicorn
uvicorn.__version__

'0.15.0'

In [10]:
import multipart
multipart.__version__

'0.0.5'

## Elección del modelo que se va a desplegar
Antes de iniciar el código de nuestra primer API usando FastAPI, recordemos el final del workshop anterior, donde exportamos el pipeline de preprocesamiento de los datos y el modelo en objetos ".dill".

Creamos un contexto usando `open()` y usamos la función `dill.dump()` para guardar nuestro flujo de preprocesamiento en disco.

```python
import dill
dill.settings['recurse'] = True
dill.dump(full_pipeline, f, protocol=dill.HIGHEST_PROTOCOL)

with open('./preprocesser.pkl', 'wb') as f:
    dill.dump(full_pipeline, f)
```

Creamos un contexto usando `open()` y usamos la función `dill.dump()` para guardar nuestro modelo en disco. El modelo que usaremos es la Regresión Lineal.

```python
with open('./lr_model.pkl', 'wb') as f:
    dill.dump(model,f)
```

Luego, también necesitamos un archivo llamado "requirements.txt".

El archivo "requirements.txt" tiene los requerimientos para la API, que se pueden observar a continuación:

```
dill==0.3.3
scikit-learn==0.24.1
python-multipart==0.0.5
numpy==1.21.0
pandas==1.2.5
```

Ahora, vamos a elegir un lugar en nuestro directorio para poner todos los archivos relacionados al despliegue del modelo. A ésta carpeta la podemos nombrar "despliegue" y los primeros archivos que almacenaremos acá serán el pipeline de procesamiento "preprocesser.pkl", el script "transformes.py" el cual es usado por el preprocesador, el modelo Regresión Linear 'lr_model.dill' y el archivo "requirements.txt".

```
despliegue
├── preprocesser.pkl
├── lr_model.pkl
├── transformers.py
└── requirements.txt
```

## Importar dependencias, Preprocesador y Modelo
1. Importaremos las dependencias en la ruta de despliegue.

In [11]:
#pip install -r /despliegue/requirements.txt

2. Vamos a crear un script en nuestra carpeta de despliegue llamado `main.py`. Y vamos a importar las  dependencias necesariar para usar FastAPI, el preprocesador y el modelo. También vamos a crear nuestra API, la cual tendrá el nombre de "Taxi Trips Duration Predictor".

```python
import numpy as np
import pandas as pd
import dill
from fastapi import FastAPI, File
from fastapi.responses import JSONResponse
from pydantic import BaseModel

from transformers import TransformerFechas, TransformerDistancia, TransformerVelocidad

app = FastAPI(title="Taxi Trips Duration Predictor")
```

3. Ahora vamos a importar el modelo y el preprocesador que creamos y exportamos en la sesión anterior.

```python
# import the preprocessor
with open("preprocesser.pkl", "rb") as f:
    preprocessor = dill.load(f)
# import the model
with open("lr_model.pkl", "rb") as f:
    model = dill.load(f)
```

## Solicitudes y Respuestas
Las API funcionan mediante “solicitudes” y “respuestas”. Cuando una API solicita información de una aplicación web o un servidor web, recibirá una respuesta. 

Nuestra API tendrá la funcionalidad de recibir "solicitudes" con los datos de los features que requiere el modelo, luego la lógica del negocio procesará los datos y finalmente la API devolverá una "respuesta" con la predicción del modelo. En este caso, la lógica del negocio consiste en el pipeline de preprocesamiento y el modelo.

## Puntos finales o "Endpoints" 
El lugar al que las API envían solicitudes y donde reside el recurso se denomina "endpoint".

Cuando una API interactúa con otro sistema, los puntos de contacto de esta comunicación se consideran puntos finales. Para las API, un punto final puede incluir una URL de un servidor o servicio. Cada punto final es la ubicación desde la que las API pueden acceder a los recursos que necesitan para llevar a cabo su función.

## Primer Endpoint: Get Endpoint
Las solicitudes GET son los métodos más comunes y más utilizados en API y sitios web. El método GET se utiliza para recuperar datos de un servidor en el recurso especificado. Por ejemplo, supongamos que tiene una API con un punto final /users. Hacer una solicitud GET a ese punto final debería devolver una lista de todos los usuarios disponibles.

`GET` es un método que puede recibir parámetros a través de parámetros "path" o parámetros "query":

- `Path Parameters`:

```python
@app.get("/{}", response_class=JSONResponse)
```

Puedes revisar la documentación de `Path Parameters` en FastAPI [acá](https://fastapi.tiangolo.com/tutorial/path-params/).

- `Query Parameters`:

```python
@app.get("/", response_class=JSONResponse)
def get_funct(
    vendor_id: int,
    pickup_datetime: str,
    passenger_count: int,
    pickup_longitude: float,
    pickup_latitude: float,
    dropoff_longitude: float,
    dropoff_latitude: float,
    pickup_borough: str,
    dropoff_borough: str,
)
```

Con los `Query Parameters`, los parámetros se definen al interior de la función `get_func()`.

Puedes revisar la documentación de `Query Parameters` en FastAPI [acá](https://fastapi.tiangolo.com/tutorial/query-params/).

En el presente ejemplo nuestro método `GET` recibirá los parámetros con la convención de query parameters y retornará la respuesta de la predicción en un JSON.

```python
import numpy as np
import pandas as pd
import dill
from fastapi import FastAPI, File
from fastapi.responses import JSONResponse

from transformers import TransformerFechas, TransformerDistancia, TransformerVelocidad

app = FastAPI(title="Taxi Trips Duration Predictor")

# import the preprocessor
with open("preprocesser.pkl", "rb") as f:
    preprocessor = dill.load(f)
# import the model
with open("lr_model.pkl", "rb") as f:
    model = dill.load(f)


@app.get("/", response_class=JSONResponse)
def get_funct(
    vendor_id: int,
    pickup_datetime: str,
    passenger_count: int,
    pickup_longitude: float,
    pickup_latitude: float,
    dropoff_longitude: float,
    dropoff_latitude: float,
    pickup_borough: str,
    dropoff_borough: str,
):
    """Serves predictions given query parameters specifying the taxi trip's
    features from a single example.

    Args:
        vendor_id (int): a code indicating the provider associated with the trip record
        pickup_datetime (str): date and time when the meter was engaged
        passenger_count (float): the number of passengers in the vehicle
        (driver entered value)
        pickup_longitude (float): the longitude where the meter was engaged
        pickup_latitude (float): the latitude where the meter was engaged
        dropoff_longitude (float): the longitude where the meter was disengaged
        dropoff_latitude (float): the latitude where the meter was disengaged
        pickup_borough (str): the borough where the meter was engaged
        dropoff_borough (str): the borough where the meter was disengaged

    Returns:
        [JSON]: model prediction for the single example given
    """
    df = pd.DataFrame(
        [
            [
                vendor_id,
                pickup_datetime,
                passenger_count,
                pickup_longitude,
                pickup_latitude,
                dropoff_longitude,
                dropoff_latitude,
                pickup_borough,
                dropoff_borough,
            ]
        ],
        columns=[
            "vendor_id",
            "pickup_datetime",
            "passenger_count",
            "pickup_longitude",
            "pickup_latitude",
            "dropoff_longitude",
            "dropoff_latitude",
            "pickup_borough",
            "dropoff_borough",
        ],
    )
    prediction = model.predict(preprocessor.transform(df))
    return {
        "features": {
            "vendor_id": vendor_id,
            "pickup_datetime": pickup_datetime,
            "passenger_count": passenger_count,
            "pickup_longitude": pickup_longitude,
            "pickup_latitude": pickup_latitude,
            "dropoff_longitude": dropoff_longitude,
            "dropoff_latitude": dropoff_latitude,
            "pickup_borough": pickup_borough,
            "dropoff_borough": dropoff_borough,
        },
        "prediction": list(prediction)[0],
    }


if __name__ == "__main__":
    import uvicorn

    # For local development:
    uvicorn.run("main:app", port=3000, reload=True)
```

Revisemos el main de nuestra API

```python
if __name__ == "__main__":
    import uvicorn

    # For local development:
    uvicorn.run("main:app", port=3000, reload=True)
```

`reload=True` permite que cada vez que se guarde el archivo main.py, la API se actualice automáticamente, sin necesidad de cerrar la API y volverla a reiniciar cada vez que se haga un cambio. Esta es una funcionalidad muy útil durante la etapa de desarrollo de la API, no es recomendable tenerla en la etapa de despligue. 

Así es como se debe ver ahora la estructura de la carpeta `despliegue`:

```python
despliegue
├── preprocesser.pkl
├── lr_model.pkl
├── transformers.py
├── main.py
└── requirements.txt
```

A continuación debemos ir a la terminal, ubicarnos en la carpeta "despliegue" y correr el siguiente comando:

```
python main.py
```

En la terminal obtendremos una salida así:

<img src='../imgs/image3.png'>

¡A continuación viene la mejor parte! ¡Vamos a probar nuestra API!

Así que ahora ve a tu buscador preferido, puede ser Chrome, Safari, ojalá no Internet Explorer 😁  si quieres acabar este workpshop hoy 🤭  

Una vez en el buscador, coloca esta dirección [`http://127.0.0.1:3000`](http://127.0.0.1:3000)

Si nos quedamos en esta dirección podríamos pasar los features del modelo editando la URL directamente, pero esto sería engorroso. Afortunadamente FastAPI tiene por default un Front que podemos usar con cualquiera de nuestras APIs. Así que sólo debemos hacer un cambio en la URL:

[`http://127.0.0.1:3000/docs`](http://127.0.0.1:3000/docs)

Ahora deberías ver algo así en el buscador:

<img src='../imgs/image4.png'>

Ahora, presionando "Try it out" puedes rellenar para cada feature su valor correspondiente, de esta forma el modelo recibe los datos que va a preprocesar y luego el modelo podrá generar su predicción y enviarla como respuesta!

<img src='../imgs/image5.png'>

**Checkpoint #1**

Try out! Ensayar el método GET con features reales:
- vendor_id: 2
- pickup_datetime: 2016-03-14 17:24:55
- passenger_count: 1
- pickup_longitude: -73.9821548462
- pickup_latitude: 40.7679367065
- dropoff_longitude: -73.964630127
- dropoff_latitude: 40.7656021118
- pickup_borough: Manhattan
- dropoff_borough: Manhattan

Al final debes obtener el codigo de respuesta éxitosa, es decir 200, y la predicción del modelo. Se debe ver más o menos así:

<img src='../imgs/image6.png'>

## Segundo Endpoint: Post Endpoint
En los servicios web, las solicitudes POST se utilizan para enviar datos al servidor API para crear o actualizar un recurso. Los datos enviados al servidor se almacenan en el cuerpo de la solicitud HTTP.

El ejemplo más simple es un formulario de contacto en un sitio web. Cuando completas las entradas en un formulario y presionas Enviar, esos datos se colocan en el cuerpo de respuesta de la solicitud y se envían al servidor. Puede ser JSON, XML o parámetros de consulta (query parameters).

En este caso, nos gustaría que nuestro método POST reciba un JSON y retorne un JSON.

`BaseModel`  de [pydantic](https://pydantic-docs.helpmanual.io/) es una clase de la cuál vamos a heredar para crear las especificaciones del JSON que va a ingresar en nuestro Endpoint. Por eso la debemos agregar a nuestras dependencias:

```python
import numpy as np
import pandas as pd
import dill
from fastapi import FastAPI, File
from fastapi.responses import JSONResponse, StreamingResponse
from pydantic import BaseModel
```

Ahora vamos a crear la clase TaxiTrip, que hereda de BaseModel. TaxiTrip va a ser un objeto con 9 parámetros que describen un viaje en taxi.

```python
class TaxiTrip(BaseModel):
    vendor_id: int
    pickup_datetime: str
    passenger_count: float
    pickup_longitude: float
    pickup_latitude: float
    dropoff_longitude: float
    dropoff_latitude: float
    pickup_borough: str
    dropoff_borough: str
```

Ahora, en nuestro `POST` endpoint vamos a definir que el parámetro que entra es un taxitrip de tipo TaxiTrip.

```python
@app.post("/json", response_class=JSONResponse)
def post_json(taxitrip: TaxiTrip):
    """Serves predictions given a request body specifying the taxis trip's features
    from a single example.

    Args:
        taxitrip (TaxiTrip): request body of type `TaxiTrip` with the
        attributes: vendor_id, pickup_datetime, passenger_count, pickup_longitude,
        pickup_latitude, dropoff_longitude, dropoff_latitude, pickup_borough and
        dropoff_borough

    Returns:
        [JSON]: model prediction for the single example given
    """
    vendor_id = taxitrip.vendor_id
    pickup_datetime = taxitrip.pickup_datetime
    passenger_count = taxitrip.passenger_count
    pickup_longitude = taxitrip.pickup_longitude
    pickup_latitude = taxitrip.pickup_latitude
    dropoff_longitude = taxitrip.dropoff_longitude
    dropoff_latitude = taxitrip.dropoff_latitude
    pickup_borough = taxitrip.pickup_borough
    dropoff_borough = taxitrip.dropoff_borough

    df = pd.DataFrame(
        [
            [
                vendor_id,
                pickup_datetime,
                passenger_count,
                pickup_longitude,
                pickup_latitude,
                dropoff_longitude,
                dropoff_latitude,
                pickup_borough,
                dropoff_borough,
            ]
        ],
        columns=[
            "vendor_id",
            "pickup_datetime",
            "passenger_count",
            "pickup_longitude",
            "pickup_latitude",
            "dropoff_longitude",
            "dropoff_latitude",
            "pickup_borough",
            "dropoff_borough",
        ],
    )
    prediction = model.predict(preprocessor.transform(df))
    return {
        "features": {
            "vendor_id": vendor_id,
            "pickup_datetime": pickup_datetime,
            "passenger_count": passenger_count,
            "pickup_longitude": pickup_longitude,
            "pickup_latitude": pickup_latitude,
            "dropoff_longitude": dropoff_longitude,
            "dropoff_latitude": dropoff_latitude,
            "pickup_borough": pickup_borough,
            "dropoff_borough": dropoff_borough,
        },
        "prediction": list(prediction)[0],
    }
```

Ahora, guardamos los cambios en el archivo `main.py` y recargamos la página:

[`http://127.0.0.1:3000/docs`](http://127.0.0.1:3000/docs)

Como podremos observar, ya nuestro `POST` Endpoint está listo para usar:

<img src='../imgs/image7.png'>

**Checkpoint #2**

Try out! Ensayar el método POST con features reales:

```python
{
  "vendor_id": 2,
  "pickup_datetime": "2016-03-14 17:24:55",
  "passenger_count": 1,
  "pickup_longitude": -73.9821548462,
  "pickup_latitude": 40.7679367065,
  "dropoff_longitude": -73.964630127,
  "dropoff_latitude": 40.7656021118,
  "pickup_borough": "Manhattan",
  "dropoff_borough": "Manhattan"
}
```

Al final debes obtener el codigo de respuesta éxitosa, es decir 200 y la predicción del modelo. Se debe ver más o menos así:

<img src='../imgs/image8.png'>

## Tercer Endpoint: POST Endpoint
Este último Endpoint será diferente al anterior ya que recibirá como entrada un archivo y devolverá un archivo a su vez. Por esto, debemos importar otro tipo de respuesta de FastAPI en nuestras dependencias, el `StreamingResponse`.

Por otro lado, nuestro método `POST` va a recibir un parámetro de tipo `File`, así que también lo debemos importar de la librería `fastapi`.

Además, debemos traer la clase `BytesIO` de la librería `io`, la cual toma un objeto bytes que está en memoria en python e imita el comportamiento de un archivo, de lo que Python considera un objeto tipo archivo, el cual tiene métodos de "read" y "write".

```python
import numpy as np
import pandas as pd
import dill
from fastapi import FastAPI, File
from fastapi.responses import JSONResponse, StreamingResponse
from pydantic import BaseModel
from io import BytesIO

from transformers import TransformerFechas, TransformerDistancia, TransformerVelocidad
```

`StreamingResponse` es muy útil ya que no guarda los archivos en disco sino que en el Browser recibe el archivo. De esta forma ahorramos espacio en el disco, evitamos las operaciones en disco y las hacemos en memoria, lo cual es mucho más rápido.

```python
@app.post("/file", response_class=StreamingResponse)
def post_file(file: bytes = File(...)):
    """Serves predictions given a CSV file with no header and seven columns
    specifying each taxi trip's features in the order vendor_id, pickup_datetime,
    passenger_count, pickup_longitude,pickup_latitude, dropoff_longitude and
    dropoff_latitude, pickup_borough and dropoff_borough

    Args:
        file (bytes, optional): bytes from a CSV file as described above.
         Defaults to File(...), but to receive a file is required.

    Returns:
        [StreamingResponse]: Returns a streaming response with a new CSV file that contains
        a column with the predictions.
    """
    # Decode the bytes as text and split the lines:
    input_lines = file.decode().splitlines()

    # Split each line as a list of the three features:
    X = [p.split(",") for p in input_lines]
    predictions = []
    for x in X:
        vendor_id = int(x[0])
        pickup_datetime = str(x[1])
        passenger_count = float(x[2])
        pickup_longitude = float(x[3])
        pickup_latitude = float(x[4])
        dropoff_longitude = float(x[5])
        dropoff_latitude = float(x[6])
        pickup_borough = str(x[7])
        dropoff_borough = str(x[8])
        df = pd.DataFrame(
            [
                [
                    vendor_id,
                    pickup_datetime,
                    passenger_count,
                    pickup_longitude,
                    pickup_latitude,
                    dropoff_longitude,
                    dropoff_latitude,
                    pickup_borough,
                    dropoff_borough,
                ]
            ],
            columns=[
                "vendor_id",
                "pickup_datetime",
                "passenger_count",
                "pickup_longitude",
                "pickup_latitude",
                "dropoff_longitude",
                "dropoff_latitude",
                "pickup_borough",
                "dropoff_borough",
            ],
        )
        # Get predictions for each taxi trip:
        prediction = model.predict(preprocessor.transform(df))
        predictions.append(prediction)

    # Append the prediction to each input line:
    output = [line + "," + str(pred[0]) for line, pred in zip(input_lines, predictions)]
    # Join the output as a single string:
    output = "\n".join(output)
    # Encode output as bytes:
    output = output.encode()

    # The kind is text, the extension is csv
    return StreamingResponse(
        BytesIO(output),
        media_type="text/csv",
        headers={"Content-Disposition": 'attachment;filename="prediction.csv"'},
    )
```

Ahora, guardamos los cambios en el archivo main.py y recargamos la página:

[`http://127.0.0.1:3000/docs`](http://127.0.0.1:3000/docs)

Como podremos observar, ya nuestro tercer Endpoint (POST) está listo para usar:

<img src='../imgs/image9.png'>

Este tercer endpoint recibe un archivo .CSV con diferentes ejemplos, te reto a generar las predicciones de los ejemplos en el siguiente archivo:

`example.csv`

**Checkpoint #3**

Try out! Ensayar el método POST con features reales del archivo CSV previo.

Al final debes obtener el codigo de respuesta éxitosa, es decir 200 y la predicción del modelo. Se debe ver más o menos así:

<img src='../imgs/image10.png'>

Como se puede observar, desde el buscador podemos descargar un archivo que contendrá los features de cada ejemplo y sus respectivas predicciones. Ese archivo se llamará "prediction.csv" y su contenido se debe ver así:

<img src='../imgs/image11.png'>

Como se puede observar la predicción fue la misma para ambos ejemplos, esto nos indica que la Regresión Lineal no es un modelo muy robusto. Para este workshop se decidió usar la Regresión Lineal por simplicidad y por ser un modelo liviano, pero lo recomendable es siempre elegir el mejor modelo que se haya encontrada en la etapa de experimentación.

# Despliegue de Software con Docker
En esta sección vamos desplegar nuestra API usando Docker.

## ¿Qué es un Contenedor de Docker?
Es una herramienta que nos permite aislar el entorno de cualquier software que queramos implementar.

Si desarrollo una pieza de software en mi entorno, con ciertas dependencias y versiones de las bibliotecas que uso en un sistema operativo específico, entonces, lo que hice en mi PC, podría no funcionar en otro PC. 

El contenedor de Docker es como un mini computador, está asociado al concepto de máquinas virtuales. Sin embargo, las máquinas virtuales asignan una gran cantidad de recursos, incluso si la aplicación no se está utilizando. 

Cuando el contenedor de Docker no usa recursos, no bloquea el host para usarlos, por lo que es más eficiente que las máquinas virtuales.

El contenedor de Docker tiene un sistema operativo específico, normalmente Linux. La aplicación se ejecuta allí, de tal manera que no importa quién la ejecute o en qué computadora, el sistema operativo y las bibliotecas operativo son los mismos.

## ¿Qué es una Imagen de Docker?
Es como un molde de donde se crean los contenedores.

Un contenedor es una instanciación específica de la imagen de Docker, que puede ejecutarse en cualquier host que tenga la aplicación de escritorio de Docker (Docker desktop).

Para seleccionar una imagen de Docker, debemos partir de una imagen que ya tenga algunas de nuestras dependencias. Por ejemplo, en este caso nos interesará una imagen que ya tenga python.

## ¿Dónde encontramos las imágenes de Docker?
Dockerhub es el repositorio oficial de imágenes de Docker

[Docker Hub Container Image Library | App Containerization](https://hub.docker.com/)

<img  src='../imgs/image12.png'>

Ahora busquemos una imagen que ya contenga FastAPI usando las palabras clave “tiangolo fastapi”:

<img src='../imgs/image13.png'>

La primera imagen que podemos ver, `tiangolo/uvicorn-gunicorn-fastapi` es lo más cercano a una “imagen oficial de Docker de fastapi” que encontraremos, ya que fue desarrollada por tiangolo, el creador de fastapi.

## ¿Qué es un Dockerfile?
Una imagen de Docker es un molde para crear un contenedor de Docker, y ¿cómo creamos ese molde? Con un Dockerfile.

Un Dockerfile es la receta para crear el molde.

Un Dockerfile es un archivo con un conjunto de pasos que le dicen a Docker cómo crear esa imagen.

Vamos a crear un archivo llamado Dockerfile, en la misma carpeta donde tenemos nuestra aplicación y luego vamos a escribir algunos comandos en él.

El directorio debería verse así ahora mismo:

```
despliegue
├── preprocesser.pkl
├── lr_model.pkl
├── transformers.py
├── main.py
├── Dockerfile
└── requirements.txt
```

## Comandos del Dockerfile
`FROM` → aquí especificamos en qué imagen nos basamos para crear nuestra imagen propia.

Ya seleccionamos la imagen base `tiangolo/uvicorn-gunicorn-fastapi` Pero también tenemos que seleccionar una versión o una etiqueta en la sección `Tags`:

<img src='../imgs/image14.png'>

Y vamos a seleccionar la etiqueta `python3.9-slim`, que es una versión liviana de python.

Entonces, el comando `FROM` se verá así:

```
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9-slim
```

`WORKDIR` → Para especificar nuestro directorio de trabajo. Ya está especificado en la imagen de Docker pero es bueno saber en que directorio de trabajo estamos.

```
WORKDIR /app
```

`EXPOSE` → El puerto que vamos a exponer. Ya está especificado en la imagen de Docker pero es bueno saber qué puerto está expuesto.

```
EXPOSE 80
```

Usamos el comando `COPY` para traer el archivo `requirements.txt` ubicado en la carpeta donde se encuentra el `Dockerfile`.

```
COPY requirements.txt .
```

`Run` → Queremos incluir algunas dependencias

`-r` →  ejecutar `pip -r`  permite recibir una ruta con una lista de dependencias que se requieran instalar. Es muy útil instalarlos todos a la vez.

No se recomienda tener varios comandos RUN en el dockerfile, es mejor tener solo un comando RUN.

`RUN` se utiliza para instalar dependencias normalmente. Para ejecutar cualquier cosa que necesitemos poner en nuestra imagen de Docker. Lo ejecutamos y termina.

```
RUN ["pip", "install", "-r","./requirements.txt"]
```

`COPY` → Necesitamos copiar cosas de nuestro host (de nuestra máquina) a la imagen de Docker. Se van a copiar en el contenedor de Docker cuando iniciemos la imagen para crear el contenedor. Esto se puede hacer con paths relativos.

Lo que sea que esté dentro de mi carpeta de `despliegue` es lo que quiero que esté en mi directorio de trabajo en el contenedor de Docker o la imagen de Docker.

`COPY .` → El 1er punto representa la carpeta donde está el Dockerfile. Entonces, en este caso, se refiere a la carpeta despliegue.

`COPY . .` → El 2ndo punto se refiere al directorio de trabajo dentro de la imagen de Docker.

Voy a obtener todo de la carpeta de despliegue, copiarlo y pegarlo todo en el directorio de trabajo de la imagen de Docker (que es la carpeta `app`).

Es una buena práctica dejar el dockerfile en la misma carpeta donde está el código de la aplicación.

```
COPY . .
```

Para los servicios que necesitamos que sigan funcionando usamos el comando `CMD`

`CMD` → Para iniciar el servicio. Por ejemplo, una aplicación que debería recibir solicitudes todo el tiempo. Este es el último comando en el dockerfile.

Ejecutaremos la aplicación con `python3`.

```
CMD ["python3","main.py"]
```

Entonces, al final, el archivo Docker se ve así:

```
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9-slim

WORKDIR /app

EXPOSE 80

COPY requirements.txt .

RUN ["pip", "install", "-r","./requirements.txt"]

COPY . .

CMD ["python3","main.py"]
```

Este es el molde de la imagen de docker que vamos a crear.

## Cambios para hacer en nuestro archivo del backend, main.py
**1. Incluir CORSMiddleware**

CORS un mecanismo para controlar quién puede acceder a nuestra API, quién puede realizar solicitudes y obtener respuestas. Tenemos que incluir esto en nuestro archivo `main.py`. Puedes consultar la documentación de fastAPI [aquí](https://fastapi.tiangolo.com/tutorial/cors/).

Un middleware es una función que se aplica a cada solicitud que llega a nuestra aplicación y también a cada respuesta. Lo cual es muy útil, por ejemplo en este caso, esta función modifica el encabezado de las solicitudes a la API para incluir información sobre qué dominios están permitidos.

En este caso, habilitaremos las solicitudes desde todas partes, agregando este fragmento de código:

```python
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
```

Sin embargo, en este caso incluir esto no es absolutamente necesario ya que esta implementación será local. Para desarrollo es útil, pero es inseguro permitir solicitudes de todas partes.

**2. Cambiar el puerto Localhost**

El puerto 3000 es localhost y deberíamos cambiarlo.

```python
if __name__ == "__main__":
    import uvicorn

    uvicorn.run("main:app", port=3000, reload=True)
```

Actualmente la aplicación intentará ejecutarse en enlocalhost en el `puerto 3000`. Eso no va a funcionar. Porque el host local del contenedor de la ventana acoplable no estará expuesto al host que ejecuta el contenedor. Incluso si lo fuera, no sería apropiado porque el puerto que exponen nuestras imágenes de la ventana acoplable es el `puerto 80`.

Tenemos que especificar un nuevo host. Por defecto es localhost. PERO para que esto sea accesible desde el exterior, incluida la máquina host que ejecuta el contenedor de Docker, el host debe ser el `“0.0.0.0”`

Cuando ejecutamos la aplicación debemos asegurarnos de que se esté ejecutando en la `IP: “0.0.0.0"` y el `Puerto: “80”`.

Así debe quedar el main del archivo:

```python
if __name__ == "__main__":
    import uvicorn

    # For local development:
    # uvicorn.run("main:app", port=3000, reload=True)

    # for docker deployment:
    uvicorn.run("main:app", host="0.0.0.0", port=80)
```

**Checkpoint #4**

Tener el Dockerfile armado y el archivo main.py modificado.

## Ahora sí podemos crear la Imagen de Docker!
**1. Crear la Imagen de Docker.**

Abre la terminal, dirigete a la carpeta de `despliegue` y corre el siguiente comando:

```
docker build -t taxi-app .
```

El `.` representa la carpeta donde se encuentra el dockerfile.

Nota: el proceso de crear la imagen de docker puede tardar unos cuantos minutos.

**2. Dirigete al Docker Desktop**

Hemos creado una nueva imagen de Docker a partir de nuestro dockerfile y podemos verla en el Docker Desktop:

<img src='../imgs/image15.png'>

Ahora podemos usar esa imagen para crear el contenedor de Docker.

**3. Crear el Contenedor de Docker**

Estamos ejecutando una imagen para crear un contenedor.

La ejecución de Docker es diferente del inicio de Docker.

- `docker run`: crea nuevos contenedores a partir de imágenes.
- `docker start`: reinicia todos los contenedores que ya se han creado.

Ahora corre en la terminal el siguiente comando:

```
docker run -p 3000:80 taxi-app
```

Debemos especificar un `"port binding"`, o un enlace de puertos.

`3000` → es el puerto en el host.

`80` → es el puerto en el contenedor. Este debería ser un puerto que exponga la imagen. Si especificamos algo diferente de `80`, la aplicación no podrá comunicarse con el host.

**4. Ahora dirigete a [http://localhost:3000/docs](http://localhost:3000/docs)**

¡Podemos observar nuestra app con éxito!

<img src='../imgs/image16.png'>

A simple vista nuestra aplicación se ve igual a cómo se veía antes, pero hemos hecho algo independiente del entorno y del sistema operativo. Funciona para mi mac y funcionará en cualquier lugar.

Cuando ejecutamos: se está ejecutando localmente pero desde su propio sistema operativo, desde un contenedor Docker que es algo similar a una VM que se ejecuta dentro de nuestra máquina. Este docker va a funcionar desde cualquier sistema operativo y cualquier entorno siempre que tengamos la aplicación Docker Desktop.

**Checkpoint #5**

Try out! Ensaya el segundo método POST con features reales:

```python
{
  "vendor_id": 2,
  "pickup_datetime": "2016-03-14 17:24:55",
  "passenger_count": 1,
  "pickup_longitude": -73.9821548462,
  "pickup_latitude": 40.7679367065,
  "dropoff_longitude": -73.964630127,
  "dropoff_latitude": 40.7656021118,
  "pickup_borough": "Manhattan",
  "dropoff_borough": "Manhattan"
}
```